In [1]:
!pip install bs4
!pip install geocoder
import urllib.request as req
from bs4 import BeautifulSoup as BS
from time import sleep
import pandas as pd
from os import getenv
import json

     |████████████████████████████████| 122kB 993kB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/a0/b0/b2/4f80b9456b87abedbc0bf2d52235414c3467d8889be38dd472
Successfully built bs4
     |████████████████████████████████| 102kB 1.0MB/s ta 0:00:011
     |████████████████████████████████| 829kB 7.3MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/d2/3d/fa76db83bf75c4f8d338c2fd15c8d33fdd7ad23a9b5e57eb6c5de26b430e/click-7.1.2-py2.py3-none-any.whl
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built future


In [2]:
import numpy as np

In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wiki_page = req.urlopen(url)
wiki_soup = BS(wiki_page, 'html.parser')
data_table = wiki_soup.find('table', class_='wikitable sortable')

headers = [x.find(text=True).rstrip() for x in data_table.findAll('th')]

df = pd.DataFrame(columns=headers)

for row in data_table.findAll('tr'):
    cells = [x.find(text=True).rstrip() for x in row.findAll('td')]
    if len(cells) != 0:
        df = df.append(pd.DataFrame([cells], columns=headers), ignore_index=True)

df.head(10)

,Postal Code,District,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


In [4]:
df = df[df.District!= 'Not assigned']

In [5]:
df.head()

,Postal Code,District,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [6]:
df.shape

(103, 3)

In [7]:
csv_geocode = 'https://cocl.us/Geospatial_data'
print(csv_geocode)

https://cocl.us/Geospatial_data


In [8]:
geocode = pd.read_csv(csv_geocode)

In [9]:
print(geocode.columns)
print(df.columns)

Index(['Postal Code', 'Latitude', 'Longitude'], dtype='object')
Index(['Postal Code', 'District', 'Neighbourhood'], dtype='object')


In [10]:
torframe = df.merge(geocode, on= 'Postal Code', how= 'left')

In [11]:
torframe = torframe[torframe['District'] == 'Downtown Toronto'].reset_index(drop=True)
torframe.head

<bound method NDFrame.head of    Postal Code          District  \
0          M5A  Downtown Toronto   
1          M7A  Downtown Toronto   
2          M5B  Downtown Toronto   
3          M5C  Downtown Toronto   
4          M5E  Downtown Toronto   
5          M5G  Downtown Toronto   
6          M6G  Downtown Toronto   
7          M5H  Downtown Toronto   
8          M5J  Downtown Toronto   
9          M5K  Downtown Toronto   
10         M5L  Downtown Toronto   
11         M5S  Downtown Toronto   
12         M5T  Downtown Toronto   
13         M5V  Downtown Toronto   
14         M4W  Downtown Toronto   
15         M5W  Downtown Toronto   
16         M4X  Downtown Toronto   
17         M5X  Downtown Toronto   
18         M4Y  Downtown Toronto   

                                        Neighbourhood   Latitude  Longitude  
0                           Regent Park, Harbourfront  43.654260 -79.360636  
1         Queen's Park, Ontario Provincial Government  43.662301 -79.389494  
2              

In [12]:
torframe

,Postal Code,District,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
5,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
6,M6G,Downtown Toronto,Christie,43.669542,-79.422564
7,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
8,M5J,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands",43.640816,-79.381752
9,M5K,Downtown Toronto,"Toronto Dominion Centre, Design Exchange",43.647177,-79.381576


In [13]:
print('The dataframe has {} district and {} neighborhoods.'.format(
        len(torframe['District'].unique()),
        torframe.shape[0]
    )
)

The dataframe has 1 district and 19 neighborhoods.


In [14]:
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.8.3
  latest version: 4.8.4

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    openssl-1.1.1g             |       h516909a_1         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.2 MB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-2.0.0

In [15]:
import json 
from pandas.io.json import json_normalize 
from sklearn.cluster import KMeans

In [16]:
address = 'Toronto'

geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 43.6534817, -79.3839347.


In [17]:
CLIENT_ID = 'EMTOJABNTMZ5ZJTS4CHKS22UD5IOTVD4C5EKSV00K4CXRBA2'
CLIENT_SECRET = 'AM4GY2YBV2IVEU2PS0NLQXZHVIQS3PTEZIMCETGVFVHXQPFN'
VERSION = '20200820'
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: Operation Complete')
print('CLIENT_SECRET: Operation Complete')

Your credentails:
CLIENT_ID: Operation Complete
CLIENT_SECRET: Operation Complete


In [18]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [19]:
import requests

In [20]:
toronto_venues = getNearbyVenues(names = torframe['Neighbourhood'], latitudes = torframe['Latitude'], longitudes = torframe['Longitude'])


print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Stn A PO Boxes
St. James Town, Cabbagetown
First Canadian Place, Underground city
Church and Wellesley
There are 210 uniques categories.


In [21]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.shape

(1250, 211)

In [26]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped.shape

(19, 211)

In [27]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [28]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
toronto_venues_sorted = pd.DataFrame(columns=columns)
toronto_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    toronto_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

toronto_venues_sorted.loc[1:18]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport Terminal,Harbor / Marina,Sculpture Garden,Airport Food Court,Airport Gate,Bar,Boat or Ferry,Coffee Shop
2,Central Bay Street,Coffee Shop,Sandwich Place,Café,Italian Restaurant,Japanese Restaurant,Salad Place,Burger Joint,Bubble Tea Shop,Poke Place,Portuguese Restaurant
3,Christie,Grocery Store,Café,Park,Coffee Shop,Diner,Nightclub,Restaurant,Baby Store,Candy Store,Italian Restaurant
4,Church and Wellesley,Coffee Shop,Japanese Restaurant,Gay Bar,Sushi Restaurant,Restaurant,Pub,Bubble Tea Shop,Smoke Shop,Café,Men's Store
5,"Commerce Court, Victoria Hotel",Coffee Shop,Café,Restaurant,Hotel,Gym,American Restaurant,Japanese Restaurant,Gastropub,Seafood Restaurant,Italian Restaurant
6,"First Canadian Place, Underground city",Coffee Shop,Café,Hotel,Gym,Restaurant,Japanese Restaurant,Seafood Restaurant,Gastropub,American Restaurant,Asian Restaurant
7,"Garden District, Ryerson",Coffee Shop,Clothing Store,Café,Cosmetics Shop,Bubble Tea Shop,Japanese Restaurant,Pizza Place,Bookstore,Middle Eastern Restaurant,Lingerie Store
8,"Harbourfront East, Union Station, Toronto Islands",Coffee Shop,Aquarium,Café,Hotel,Restaurant,Brewery,Fried Chicken Joint,Scenic Lookout,Pizza Place,Italian Restaurant
9,"Kensington Market, Chinatown, Grange Park",Café,Coffee Shop,Vegetarian / Vegan Restaurant,Bar,Mexican Restaurant,Dessert Shop,Vietnamese Restaurant,Gaming Cafe,Park,Pizza Place
10,"Queen's Park, Ontario Provincial Government",Coffee Shop,College Cafeteria,Yoga Studio,Smoothie Shop,Sandwich Place,Café,Portuguese Restaurant,Chinese Restaurant,Persian Restaurant,College Auditorium


In [29]:
for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.09
1            Beer Bar  0.04
2                 Pub  0.04
3          Restaurant  0.04
4  Seafood Restaurant  0.04
5         Cheese Shop  0.04
6        Cocktail Bar  0.04
7              Bakery  0.04
8                Café  0.04
9      Farmers Market  0.04


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                 venue  freq
0       Airport Lounge  0.13
1      Airport Service  0.13
2     Airport Terminal  0.13
3              Airport  0.07
4      Harbor / Marina  0.07
5        Boat or Ferry  0.07
6     Sculpture Garden  0.07
7                  Bar  0.07
8  Rental Car Location  0.07
9          Coffee Shop  0.07


----Central Bay Street----
                   venue  freq
0            Coffee Shop  0.18
1                   Café  0.05
2         Sandwich Place  0.05
3     Italian Restaurant  0.05
4           Burger Joint  0.03
5            Salad Pl

In [39]:
toronto_venues_sorted.loc[[2],:]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Central Bay Street,Coffee Shop,Sandwich Place,Café,Italian Restaurant,Japanese Restaurant,Salad Place,Burger Joint,Bubble Tea Shop,Poke Place,Portuguese Restaurant


In [31]:
neighbourhood_latitude = torframe.loc[5, 'Latitude'] # neighborhood latitude value
neighbourhood_longitude = torframe.loc[5, 'Longitude'] # neighborhood longitude value

neighbourhood_name = torframe.loc[5, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))

Latitude and longitude values of Central Bay Street are 43.6579524, -79.3873826.


In [32]:
import folium
from folium.plugins import FastMarkerCluster

In [33]:
# create map of Manhattan using latitude and longitude values
downtown_map = folium.Map(location=[latitude, longitude], zoom_start=15)

# add markers to map
for lat, lng, label in zip(torframe['Latitude'], torframe['Longitude'], torframe['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        popup=label,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.7,
        parse_html=False).add_to(downtown_map)  
    
downtown_map

In [34]:
downtown_map.save(outfile = 'downtownmap.html')

In [35]:
newtorframe = toronto_venues_sorted.loc[[3,4,5,6,9,11,14,15,16],:]
newtorframe

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Christie,Grocery Store,Café,Park,Coffee Shop,Diner,Nightclub,Restaurant,Baby Store,Candy Store,Italian Restaurant
4,Church and Wellesley,Coffee Shop,Japanese Restaurant,Gay Bar,Sushi Restaurant,Restaurant,Pub,Bubble Tea Shop,Smoke Shop,Café,Men's Store
5,"Commerce Court, Victoria Hotel",Coffee Shop,Café,Restaurant,Hotel,Gym,American Restaurant,Japanese Restaurant,Gastropub,Seafood Restaurant,Italian Restaurant
6,"First Canadian Place, Underground city",Coffee Shop,Café,Hotel,Gym,Restaurant,Japanese Restaurant,Seafood Restaurant,Gastropub,American Restaurant,Asian Restaurant
9,"Kensington Market, Chinatown, Grange Park",Café,Coffee Shop,Vegetarian / Vegan Restaurant,Bar,Mexican Restaurant,Dessert Shop,Vietnamese Restaurant,Gaming Cafe,Park,Pizza Place
11,"Regent Park, Harbourfront",Coffee Shop,Park,Pub,Bakery,Café,Breakfast Spot,Theater,Cosmetics Shop,Shoe Store,Restaurant
14,St. James Town,Coffee Shop,Café,Cocktail Bar,American Restaurant,Clothing Store,Restaurant,Moroccan Restaurant,Cosmetics Shop,Lingerie Store,Beer Bar
15,"St. James Town, Cabbagetown",Coffee Shop,Restaurant,Pizza Place,Café,Chinese Restaurant,Pub,Bakery,Convenience Store,Park,Pet Store
16,Stn A PO Boxes,Coffee Shop,Pub,Café,Japanese Restaurant,Restaurant,Italian Restaurant,Beer Bar,Seafood Restaurant,Hotel,Creperie


In [40]:
address = '138 W 72nd St, Upper West Side, New York City, NY'

geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.77777945, -73.98033926023382.


In [42]:
# create map of Manhattan using latitude and longitude values
downtown_map = folium.Map(location=[latitude, longitude], zoom_start=15)

# add markers to map
for lat, lng, label in zip(torframe['Latitude'], torframe['Longitude'], torframe['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        popup=label,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.7,
        parse_html=False).add_to(downtown_map)  
    
downtown_map

In [43]:
neighbourhood_latitude = torframe.loc[5, 'Latitude'] # neighborhood latitude value
neighbourhood_longitude = torframe.loc[5, 'Longitude'] # neighborhood longitude value

neighbourhood_name = torframe.loc[5, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))

Latitude and longitude values of Central Bay Street are 43.6579524, -79.3873826.


In [49]:
map_cbs = folium.Map(location=[43.6579524, -79.3873826], zoom_start=18)

folium.features.CircleMarker(
    [latitude, 
    longitude],
    radius=20,
    color='red',
    popup='2,000 sqft',
    fill = True,
    fill_color = 'Blue',
    fill_opacity = 0.8
).add_to(map_cbs) 
    
map_cbs